In [2]:
import polars as pl

In [4]:

area = "hanoi"

df = pl.read_parquet(f"./datasets/raw/oss/{area}_*.parquet").unique()
# df.write_parquet(f"./datasets/results/{area}.parquet")
len(df)

ComputeError: expected at least 1 source

This error occurred with the following context stack:
	[1] 'parquet scan'
	[2] 'sink'


In [15]:
df[0]["address"].to_list()

['28 Xuân Tảo, Tây Hồ, Hà Nội 100000']

In [17]:
areas = ["an_giang","bac_giang","bac_lieu","binh_dinh","binh_duong","binh_phuoc","binh_thuan","ca_mau","can_tho","cao_bang","dak_nong","da_nang","dong_nai","gia_lai","ha_giang","ha_nam","hau_giang","hcm","hoa_binh","hung_yen","khanh_hoa","kien_giang","kon_tum","lai_chau","lam_dong","lang_son","long_an","nghe_an","ninh_binh","ninh_thuan","phu_yen","quang_nam","quang_ngai","quang_ninh","quang_tri","soc_trang","tay_ninh","thai_nguyen","thanh_hoa","tien_giang","tra_vinh","tuyen_quang","vinh_long","vung_tau","yen_bai","hue","bac_kan","bac_ninh","ben_tre","dak_lak","dien_bien","dong_thap","hai_duong","hai_phong","ha_noi","ha_tinh","lao_cai","nam_dinh","phu_tho","quang_binh","quang_ngai","son_la","thai_binh","vinh_phuc"]

for area in areas:
    df = pl.read_parquet(f"./datasets/raw/oss/{area}_*.parquet").unique()
    df.write_parquet(f"./datasets/results/{area}.parquet")
    print(area, len(df))

an_giang 12946
bac_giang 4711
bac_lieu 2484
binh_dinh 5523
binh_duong 14709
binh_phuoc 5654
binh_thuan 5276
ca_mau 2702
can_tho 4077
cao_bang 1432
dak_nong 3491
da_nang 6352
dong_nai 20876
gia_lai 5175
ha_giang 2113
ha_nam 7249
hau_giang 5379
hcm 27428
hoa_binh 3547
hung_yen 5401
khanh_hoa 5275
kien_giang 7388
kon_tum 1363
lai_chau 1142
lam_dong 7339
lang_son 1731
long_an 8490
nghe_an 16773
ninh_binh 3911
ninh_thuan 2427
phu_yen 3991
quang_nam 6756
quang_ngai 3285
quang_ninh 5941
quang_tri 2379
soc_trang 3587
tay_ninh 3453
thai_nguyen 4647
thanh_hoa 7406
tien_giang 13581
tra_vinh 3360
tuyen_quang 2156
vinh_long 2935
vung_tau 4622
yen_bai 1538
hue 3466
bac_kan 867
bac_ninh 5807
ben_tre 4286
dak_lak 7952
dien_bien 1080
dong_thap 10359
hai_duong 7067
hai_phong 5024
ha_noi 28725
ha_tinh 5261
lao_cai 2267
nam_dinh 5709
phu_tho 7611
quang_binh 3460
quang_ngai 3285
son_la 2163
thai_binh 5404
vinh_phuc 4723


In [130]:
from mylib import ALL_TYPES, utils, POI_GROUPS, viz

df = pl.read_parquet("./datasets/raw/oss/summary/*.parquet", allow_missing_columns=True)
new_df = df.unique().with_columns(is_poi_transport = pl.col("query").is_in(POI_GROUPS['group1']).cast(pl.Int64))\
            .with_columns(is_poi_ecom = pl.col("query").is_in(POI_GROUPS['group2']).cast(pl.Int64))\
            .with_columns(is_poi_popu = pl.col("query").is_in(POI_GROUPS['group3']).cast(pl.Int64))\
            .with_columns(is_ATM = (pl.col("categories").str.contains('(atm)|(ATM)') | pl.col('query').eq('atm')).cast(pl.Int64))\
            .with_columns([
                    pl.arange(0, df.height).alias("id")
                ])



# new_df.select(pl.col("is_ATM").sum())
new_df.write_parquet("./datasets/results/vietnam.parquet")

In [49]:
df.filter(pl.col("query").eq("bank")).select(pl.col("categories")).unique().to_series().to_list()
df.filter(pl.col("query").eq("atm")).select(pl.col("categories")).unique().to_series().to_list()

['Loan agency',
 'Bank, ATM, Financial consultant',
 'Bank, Savings bank',
 'Jewelry store, Crypto ATM, Gold dealer, Money transfer service',
 'Credit union',
 'Gold dealer',
 'Banking and finance',
 'Bank, Currency exchange service',
 'Bank, ATM',
 'Mortgage lender',
 'Bank',
 'Financial institution',
 'ATM',
 'Financial institution, ATM, Bank, Financial consultant',
 'ATM, Bank',
 'Bank, Investment bank, Investment service, Money transfer service',
 'Financial institution, Bank, Financial consultant',
 'Loan agency, Insurance broker',
 'Bank, Mortgage broker',
 'Savings bank',
 'Financial institution, Insurance agency']

In [133]:
df = pl.read_parquet("./datasets/results/vietnam.parquet")
result = df.drop(pl.col("query","link", "categories", "complete_address")).rename({'title': 'name'}).unique()
result

name,category,latitude,longitude,address,is_poi_transport,is_poi_ecom,is_poi_popu,is_ATM,id
str,str,f64,f64,str,i64,i64,i64,i64,i64
"""Cafe Thái Nhân""","""Coffee shop""",10.510209,105.562809,"""217 QL30, Phong Mỹ, Cao Lãnh, …",0,1,0,0,253559
"""Cửa hàng điện lạnh Duy Lanh""","""Appliance store""",20.361117,106.13691,"""Xóm hát, Thành Lợi, Vụ Bản, Na…",0,1,0,0,336616
"""Thaco Auto Peugeot - TP. HCM (…","""Peugeot dealer""",10.719417,106.606578,"""75 Đ. Võ Văn Kiệt, An Lạc, Bìn…",0,1,0,0,16437
"""Tàu lượn siêu tốc vườn thú Hà …","""Roller coaster""",21.0297,105.804343,"""Đường Bưởi, Thủ Lệ, Ba Đình, H…",1,1,0,0,255015
"""168 lau""","""Chinese restaurant""",21.151662,106.104318,"""Bắc Ninh, Tp. Bắc Ninh, Vân Dư…",0,1,0,0,374285
…,…,…,…,…,…,…,…,…,…
"""Saffron VIỆT NAM - Tổng Đại Lý…","""Natural goods store""",11.075738,106.786033,"""Đường số 5, khu dân cư Genimex…",1,1,0,0,230221
"""Quầy Thuốc Hồng Vân Chợ Mới""","""Pharmacy""",20.72821,105.797667,"""Phương Tú, Ứng Hòa, Hà Nội""",0,1,0,0,174150
"""Nắng Coffee (Cà phê rang xay)""","""Cafe""",21.584979,105.846171,"""52 Đường Hoàng Ngân, Phan Đình…",0,1,0,0,345102


In [116]:
import re
def extract(url: str) -> str:
    pattern = r"19s(.+)\?authuser"  # Matches one or more digits
    # text = "'https://www.google.com/maps/place/Trung+h%E1%BB%8Dc+C%C6%A1+s%E1%BB%9F+H%C3%B2a+B%C3%ACnh+Th%E1%BA%A1nh+%28c%C6%A1+s%E1%BB%9F+1%29/data=!4m7!3m6!1s0x310a0deaf9801691:0x7749c99a91c1f71e!8m2!3d10.4202561!4d105.3452758!16s%2Fg%2F11ljcy_zb3!19sChIJkRaA-eoNCjERHvfBkZrJSXc?authuser=0&hl=en&rclk=1'"
    
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    else:
        return url

l = df.select(pl.col("link")).to_series().to_list()[:1000]

for url in l:
    print(extract(url))

ChIJkRaA-eoNCjERHvfBkZrJSXc
ChIJVZAqCgARCjERw9Q-qiZX_TM
ChIJxaC_4_EdCjERr1m2Sodd7hE
ChIJVVVVVQX7CTERdbSp2K4_Wj8
ChIJ-aFLFQD1CTERKm1si9Lw8NA
ChIJyWYfNcgXCjERqKZB70DYQzs
ChIJNzCJTDMjCjERjKEiMy1tPb8
ChIJM2B4CCkTCjER5RvPq5kjM50
ChIJ8-IYSGMPCjERJOG3HbBA6fE
ChIJGeMX0PJrCjERhsDGlXb1FRE
ChIJqQvow2IgCjERqdZbZSZoEqI
ChIJ42ea396poDERD3RIh-pGRjE
ChIJYSzQSZUTCjERlgpCvMeTTik
ChIJfWL_pakRCjEROpS4bcIa6vw
ChIJB0BjYi5vCjERiQoNP2ONGg8
ChIJZ69LkCEhCjERLeSk1SkjEzY
https://www.google.com/maps/place/An+Giang/@10.4235326,104.9861673,18z/data=!4m6!3m5!1s0x310a198a4ee22a8d:0xdc1cb02c52c088ba!8m2!3d10.5215836!4d105.1258955!16zL20vMDNxNXo5?hl=en&entry=ttu&g_ep=EgoyMDI1MDQzMC4xIKXMDSoASAFQAw%3D%3D
ChIJ4-ypuo_3CTERquDP-anuYF8
ChIJDW4gCF5zCjER9gHkTKBB-6Q
ChIJfReWhPlyCjER9y759H0Czss
ChIJVVVVVUFyCjER_UooRpRLsGE
ChIJCU4boYwHCjERyffhqMelLp8
ChIJd5R7o82KCTERQJY2g2tJM_E
ChIJ4xOeLNUjCjERWGWeZK4YC5M
ChIJEaxkPpYNCjERunZTVPgAynQ
https://www.google.com/maps/place/Ph%C3%A0+An+H%C3%B2a/@10.3892365,105.436214,18z/data=!4m6!3m5!1s